# Pure CNN using VGG16

This notebook is a simple CNN using VGG16 architecture.

In [1]:
# Libraries import
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.transforms import functional as F
import numpy as np
import os
import json
from PIL import Image
from pycocotools.coco import COCO


In [7]:
# Colab setup
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')

    !mkdir -p ./Datasets
    # copy dataset to local dir
    !cp /content/drive/MyDrive/Datasets/YOLO_UODD_dataset.zip ./Datasets/YOLO_UODD_dataset.zip
    !unzip -q ./Datasets/YOLO_UODD_dataset.zip -d ./Datasets

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import CocoDetection

class UODDDataset(Dataset):
    def __init__(self, root, annotation_file, transforms=None):
        self.root = root
        self.transforms = transforms
        with open(annotation_file) as f:
            self.coco = json.load(f)
        self.ids = list(sorted(self.coco['images'], key=lambda x: x['id']))
        # Filter images with annotations
        self.ids = [img for img in self.ids if self.has_annotations(img['id'])]

    def has_annotations(self, img_id):
        ann_ids = [ann for ann in self.coco['annotations'] if ann['image_id'] == img_id]
        return len(ann_ids) > 0

    def __getitem__(self, idx):
        img_id = self.ids[idx]['id']
        img_info = self.ids[idx]
        img_path = os.path.join(self.root, img_info['file_name'])
        img = Image.open(img_path).convert("RGB")

        ann_ids = [ann for ann in self.coco['annotations'] if ann['image_id'] == img_id]
        boxes = []
        labels = []
        for ann in ann_ids:
            xmin, ymin, w, h = ann['bbox']
            boxes.append([xmin, ymin, xmin + w, ymin + h])
            labels.append(ann['category_id'])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)

        target = {"boxes": boxes, "labels": labels}

        if self.transforms is not None:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.ids)


def get_transform():
    transforms = []
    transforms.append(F.to_tensor)
    return torchvision.transforms.Compose(transforms)

# Cell 5: Load data
train_data = UODDDataset('./Datasets/YOLO_UODD/images/train', './Datasets/YOLO_UODD/annotations/coco/instances_train.json', transforms=get_transform())
val_data = UODDDataset('./Datasets/YOLO_UODD/images/val', './Datasets/YOLO_UODD/annotations/coco/instances_val.json', transforms=get_transform())


train_loader = DataLoader(train_data, batch_size=8, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(val_data, batch_size=8, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

In [15]:
print(f"Train dataset: {len(train_data)}")
print(f"Validation dataset: {len(val_data)}")

Train dataset: 2558
Validation dataset: 128


In [16]:
from torchvision.models import VGG16_Weights

backbone = torchvision.models.vgg16(weights=VGG16_Weights.DEFAULT).features
backbone.out_channels = 512

# Define the RPN anchor generator
rpn_anchor_generator = AnchorGenerator(
    sizes=((32, 64, 128, 256, 512),),
    aspect_ratios=((0.5, 1.0, 2.0),) * 1
)

# Define the RoI pooler
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'], output_size=7, sampling_ratio=2)

# Create the Faster R-CNN model
model = FasterRCNN(backbone, num_classes=5,
                   rpn_anchor_generator=rpn_anchor_generator,
                   box_roi_pool=roi_pooler)

In [17]:
from tqdm import tqdm

# Training setup
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Training function with progress bar and loss printing
def train_one_epoch(model, optimizer, data_loader, device, epoch):
    model.train()
    running_loss = 0.0
    num_batches = len(data_loader)
    progress_bar = tqdm(data_loader, desc=f"Epoch {epoch+1}", unit="batch")

    for images, targets in progress_bar:
        images = list(img.to(device) for img in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        running_loss += losses.item()

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        progress_bar.set_postfix(loss=losses.item())

    epoch_loss = running_loss / num_batches
    print(f"Epoch {epoch+1} Loss: {epoch_loss:.4f}")

# Training loop with tqdm progress bar
num_epochs = 2
for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, train_loader, device, epoch)
    lr_scheduler.step()

# Save the model
torch.save(model.state_dict(), 'fasterrcnn_vgg16_uodd.pth')

print("Training complete. Model saved as 'fasterrcnn_vgg16_uodd.pth'.")

Epoch 1: 100%|██████████| 320/320 [02:08<00:00,  2.50batch/s, loss=0.293]


Epoch 1 Loss: 0.4504


Epoch 2: 100%|██████████| 320/320 [02:05<00:00,  2.55batch/s, loss=0.301]


Epoch 2 Loss: 0.3366
Training complete. Model saved as 'fasterrcnn_vgg16_uodd.pth'.


In [18]:
if 'google.colab' in str(get_ipython()):
  !cp ./fasterrcnn_vgg16_uodd.pth /content/drive/MyDrive/Models/fasterrcnn_vgg16_uodd.pth

In [20]:
import torch
import torchvision
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np
import json


val_root = './Datasets/YOLO_UODD/images/val'
val_annFile = './Datasets/YOLO_UODD/annotations/coco/instances_val.json'
val_dataset = UODDDataset(val_root, val_annFile, transforms=get_transform())
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=4, collate_fn=lambda x: tuple(zip(*x)))

# Function to evaluate the model
def evaluate(model, data_loader, device):
    model.eval()
    coco = COCO(val_annFile)
    results = []

    with torch.no_grad():
        for images, targets in tqdm(data_loader, desc="Evaluating"):
            images = list(img.to(device) for img in images)
            outputs = model(images)

            for target, output in zip(targets, outputs):
                image_id = target["image_id"].item()
                boxes = output["boxes"].cpu().numpy()
                scores = output["scores"].cpu().numpy()
                labels = output["labels"].cpu().numpy()

                for box, score, label in zip(boxes, scores, labels):
                    result = {
                        "image_id": image_id,
                        "category_id": label,
                        "bbox": box.tolist(),
                        "score": score
                    }
                    results.append(result)

    # Save results in COCO format
    with open("results.json", "w") as f:
        json.dump(results, f)

    coco_dt = coco.loadRes("results.json")
    coco_eval = COCOeval(coco, coco_dt, iouType='bbox')

    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    # Extract metrics
    precision = coco_eval.stats[0]  # Precision @ IoU=0.50
    recall = coco_eval.stats[1]     # Recall @ IoU=0.50
    mAP_50 = coco_eval.stats[1]     # mAP @ IoU=0.50
    mAP_50_95 = coco_eval.stats[0]  # mAP @ IoU=0.50:0.95

    return precision, recall, mAP_50, mAP_50_95

# Evaluate the model
precision, recall, mAP_50, mAP_50_95 = evaluate(model, val_loader, device)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"mAP@50: {mAP_50:.4f}")
print(f"mAP@50-95: {mAP_50_95:.4f}")

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


Evaluating:   0%|          | 0/64 [00:00<?, ?it/s]


KeyError: 'image_id'